In [1]:
import itertools
import json
import matplotlib.pyplot as plt
%matplotlib inline
import networkx as nx
import numpy as np
import pandas as pd
import random 
import re
import string
import warnings
warnings.simplefilter("ignore")

from collections import Counter
import gensim
from gensim import corpora
from gensim.models.word2vec import Word2Vec
from gensim.models import Phrases
from gensim.models.phrases import Phraser
from gensim.models import CoherenceModel

from konlpy.utils import pprint
from konlpy.tag import Hannanum
hannanum = Hannanum()
from konlpy.tag import Okt
okt = Okt()
from konlpy.tag import Kkma
kkma = Kkma()


from pandas import read_excel
from sklearn.feature_extraction.text import CountVectorizer
from tqdm import tqdm
from tqdm import trange
from wordcloud import WordCloud

from msba import posts as p
from msba import stopwords as stopwords

##########################################################

def interested_words():
    # 핵심단어 읽어 오기
    my_sheet = '소비키워드'
    keywords_filename = 'deskresearch_.xlsx'
    df = read_excel(keywords_filename, sheet_name = my_sheet, header=1) # index_col='번호'
    keywords = df['핵심단어']
    subkeywords = df['대체어']
    interested_words = df['키워드']
    return keywords, subkeywords, interested_words

def oneDArray(x):
    return list(itertools.chain(*x))

def preprocessing(text):
    # 개행문자 제거
    text = re.sub('\\\\n', ' ', text)
#     text = re.sub('[A-z]', '', text)
#     text = re.sub('[0-9]', '', text)
    text = re.sub('[\'\"]', '', text)
    text = re.sub('[\'\n"]', '', text)
    res = ''.join([i for i in text if not i.isdigit()]) 
    return text

def preprocessing_2(rows):
    pattern = re.compile(r"[.,?!★~]")
    sentences = []
    for row in rows:
        sentences = sentences + pattern.split(row.replace("\xa0", "").replace("\t","").strip())
        sentences = list(set(sentences))        
    return sentences

def convert_list_to_string(org_list, seperator=' '):
    """ Convert list to string, by joining all item in list with given separator.
        Returns the concatenated string """
    return seperator.join(org_list)

# 실전 : 말뭉치 생성 및 전처리
def preprocessing_3(sentences):
    # Convert list of strings to string
#     full_str = convert_list_to_string(sentences[1])
#     # print(full_str)

    pattern = re.compile(r".*(광고정보).*")

    corpus = []
    for sentence in sentences:
        if len(sentence):
            sentence = sentence.strip().split(" ")
            sentence = convert_list_to_string(sentence).replace("  ", " ").strip()
            if pattern.match(sentence):
                continue
            else:
                corpus.append(sentence)
    return corpus

def preprocessing_4(docs):
    # docs = [
    #         w for w in hannanum.nouns(" ".join(cell)) for cell in cells
    #         if ((not w[0].isnumeric()) and (w[0] not in string.punctuation))
    # ]
    vect = CountVectorizer(stop_words=stopwords.stopwords_kr, min_df=5, max_df=200).fit(docs)
    count = vect.transform(docs).toarray().sum(axis=0)
    idx = np.argsort(-count)
    count = count[idx]
    feature_name = np.array(vect.get_feature_names())[idx]
    # plt.bar(range(len(count)), count)
    # plt.show()

    tf_list = list(zip(feature_name, count))[:100]

    tf_df = pd.DataFrame(tf_list,columns=['단어', '빈도'])
    return tf_df

def remove_stopwords_from_list(lst):
    result = []
    for w in lst:
        if ((len(w) > 1) and (not w.isdigit())):
            if w not in stopwords.stopwords_kr: 
                result.append(w)
    return result

def remove_stopwords(line):
    result = ""
    for w in line.split(" "):
        if ((len(w) > 1) and (not w.isdigit())):
            if w not in stopwords.stopwords_kr: 
                result = result + w + " "
    return result.strip()

def getTopics(model):
    topics = []
    for topic in model.print_topics(num_words=500):
        i=1
        model_words=[]
        topic_words=str(topic).split('"')
        for words in topic_words:
            if i%2==0:
                model_words.append(words)
            i+=1
        topics.append(model_words)
    return topics
    
def save_to_csv(output, keyword, type, tf_df):
    # save to csv    
    filename = "./output/" + output + "_" + type + "_" + keyword.replace(" ","") + ".csv"   
    # filename_list.append(filename)
    tf_df.to_csv(filename, date_format='%Y%m%d', encoding='utf-8-sig')

In [2]:
## 키워드/관심어 불러오기
keywords, subkeywords, interested_words = interested_words()

for keyword, subkeyword, interested_word in zip(keywords, subkeywords, interested_words):
    subkeyword = subkeyword.replace(" ", "").replace(",","|")
    interested_word = subkeyword + "|" + interested_word.replace(" ", "").replace(",","|")

## 형태소 분석

for keyword, subkeyword, interested_word in tqdm(zip(keywords, subkeywords, interested_words)):

    #keyword = '1인 외식'
    keyword = keyword.replace(" ","")
    df = p.readall(keyword.replace(" ",""))
    df = df[ (df['date'] >= '2019-07-01') & (df['date'] < '2020-07-01')]
    df = df.drop_duplicates()
    # print(df.shape)  
    rows = df['title'].apply(preprocessing) + df['content'].apply(preprocessing)
    rows_date = df['date']

    sentences = preprocessing_2(rows)
    # print(len(sentences), type(sentences))
    sentences = preprocessing_3(sentences)
    # print(len(corpus), type(corpus))

    # 4. 각 문장별로 형태소 구분하기
    sentences_tag = []
    for sentence in sentences:
        morph = okt.pos(sentence)
        sentences_tag.append(morph)

    # 5. 명사 혹은 형용사인 품사만 선별해 리스트에 담기
    posts = []
    for sentence1 in sentences_tag:
        words = ""
        for word, tag in sentence1:
            if tag in ['Noun']:     # 명사만 추출, # 명사/형용사 추출은 ['Noun','Adjective']
                words = words + word + " "
    #6 불용어 제거
        words = remove_stopwords(words)
        posts.append(words)

    # print(len(posts), type(posts))
    # print(len(rows_date), type(rows))

    #7 공백라인 및 NaN 제거
    while("" in posts) : 
        posts.remove("") 
    while("NaN" in posts) : 
        posts.remove("") 
    # posts

    ## 최빈어

    tf_list = []
    for sentence1 in posts:
        sentence1.split()
        tf_list.append(sentence1.split())

    lst = oneDArray(tf_list)

    # 6. 선별된 품사별 빈도수 계산 & 상위 빈도 10위 까지 출력
    counts = Counter(lst)

    toplist = counts.most_common(500)
    # type(toplist)

    tf_df = pd.DataFrame (toplist,columns=['단어', '빈도수'])
    # tf_df
    #save_to_csv(keyword, "최빈어", tf_df)
    save_to_csv("TM", keyword, "최빈어", tf_df)

    ## 토픽 모델링

    dataset = pd.DataFrame(posts, columns=['문장'])

    # nan_value = float("NaN")
    # dataset.replace("", nan_value, inplace=True)
    # dataset.dropna(subset = ["문장"], inplace=True)
    # dataset.reindex

    tmp_corpus = dataset['문장'].map(lambda x: x.split('.'))

    #tmp_corpus

    # type(tmp_corpus)
    # tmp_corpus[11]
    # tmp_corpus[13]

    # corpus [[w1,w2,w3..],[..]]
    corpus = []
    for i in range(len(tmp_corpus)):
        for line in tmp_corpus[i]:
            words = [x for x in line.split()]
            corpus.append(words)
    #   corpus

    num_of_sentences = len(corpus)
    num_of_words = 0
    for line in corpus:
        num_of_words += len(line)

    #print('Num of sentences - %s'%(num_of_sentences))
    #print('Num of words - %s'%(num_of_words))

    ### Gensim

    nouns = corpus
    bigram = gensim.models.Phrases(nouns)
    trigram = gensim.models.Phrases(bigram[nouns])
    bigram_model = gensim.models.phrases.Phraser(bigram)
    trigram_model = gensim.models.phrases.Phraser(trigram)
    
    bigram_document = [bigram_model[nouns] for nouns in nouns]
    # bigram_document[1]
    # bigram_document[3]

    id2word = corpora.Dictionary(bigram_document)
    corpus = [id2word.doc2bow(doc) for doc in bigram_document]
    # corpus[0]

    # corpus[7]  # 단어별 출현 출현 수

    ### Topic Coherence 계산

    # 토픽 모델링을 수행함에 있어서 적절한 토픽의 갯수를 찾는 것이 중요하다. 
    # 2부터 9까지 값을 늘려가면서 LDA 모델을 생성하여 각 모델의 coherence를 계산한다.
    # 그리고, 적절한 토픽의 수는 토픽 갯수를 늘려가며 높은 coherence score 를 가지는 값으로 결정한다

    coherence_score=[]
    for i in range(2,10):
        model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=i)
        coherence_model = CoherenceModel(model, texts=bigram_document, dictionary=id2word, coherence='c_v')
        coherence_lda = coherence_model.get_coherence()
        print('n=',i,'\nCoherence Score: ', coherence_lda)
        coherence_score.append(coherence_lda)

#     k=[]
#     for i in range(2,10):
#         k.append(i)

#     x=numpy.array(k)
#     y=numpy.array(coherence_score)
#     title = f'{keyword} Topic Coherence'
#     plt.title(title)
#     plt.plot(x,y)
#     plt.xlim(2,10)
#     plt.xlabel('Number Of Topic (2-10)')
#     plt.ylabel('Cohrence Score')
#     plt.show()

    model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=3)
    # model.print_topics()
    model.print_topics(num_words=500) # num_topics=20

    _topics = getTopics(model)
    #_topics

    #type(_topics)

    co_top_df = pd.DataFrame(_topics)

    co_top_df.set_index(0)

    save_to_csv("TM", keyword, "토픽", co_top_df)

0it [00:00, ?it/s]

n= 2 
Coherence Score:  0.20262752272469609
n= 3 
Coherence Score:  0.2741304707305362
n= 4 
Coherence Score:  0.2552944279900016
n= 5 
Coherence Score:  0.2821508944066429
n= 6 
Coherence Score:  0.36027996572187754
n= 7 
Coherence Score:  0.33498464273357137
n= 8 
Coherence Score:  0.3647415711027213
n= 9 
Coherence Score:  0.36067892144201674


1it [03:29, 209.62s/it]

n= 2 
Coherence Score:  0.24939763701780301
n= 3 
Coherence Score:  0.2758154228352982
n= 4 
Coherence Score:  0.3469035239513858
n= 5 
Coherence Score:  0.32187985560715665
n= 6 
Coherence Score:  0.3437432803644925
n= 7 
Coherence Score:  0.359767844119183
n= 8 
Coherence Score:  0.36990209336398794
n= 9 
Coherence Score:  0.3531898126232298


2it [07:48, 224.25s/it]

...

33it [1:48:44, 281.14s/it]

n= 2 
Coherence Score:  0.3501215798430394
n= 3 
Coherence Score:  0.3401687515525084
n= 4 
Coherence Score:  0.35596149238851343
n= 5 
Coherence Score:  0.36094114819146617
n= 6 
Coherence Score:  0.43610938232423363
n= 7 
Coherence Score:  0.4014240086969943
n= 8 
Coherence Score:  0.4431008293380983
n= 9 
Coherence Score:  0.42489211086982503


34it [1:50:40, 231.67s/it]

n= 2 
Coherence Score:  0.5048983047123573
n= 3 
Coherence Score:  0.46930606736208397
n= 4 
Coherence Score:  0.5489640233801534
n= 5 
Coherence Score:  0.5141907514985677
n= 6 
Coherence Score:  0.49512703303551603
n= 7 
Coherence Score:  0.4690759518093249
n= 8 
Coherence Score:  0.44987827997385715
n= 9 
Coherence Score:  0.4452292126954551


35it [1:52:48, 193.38s/it]
